In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터

In [2]:
var_name = ['Temp', 'Wind', 'Moisture', 'Precipitation', 'Sun']
target = 'Wh'

In [3]:
def parsing(data: pd.DataFrame):
    x = data.iloc[:, 3:8]
    x.columns = var_name
    y = data.iloc[:, 2]
    y.name = target
    
    return x, y

In [4]:
def load_data():
    data = pd.read_csv('C:/Users/sec/Desktop/DSL/train.csv', encoding='cp949')
    data = data[: 100]
    x, y = parsing(data)    
    cut = int(round(len(x)*0.88))
    tr_x = x[:cut]
    tr_y = y[:cut]
    te_x = x[cut:]
    te_y = y[cut:]
    return tr_x, tr_y, te_x, te_y

In [47]:
train_x, train_y, test_x, test_y = load_data()

## 데이터 확인

In [6]:
print(train_x.head()) 
print(train_y.head())

   Temp  Wind  Moisture  Precipitation  Sun
0  17.6   2.5      92.0            0.8  0.0
1  17.7   2.9      91.0            0.3  0.0
2  17.5   3.2      91.0            0.0  0.0
3  17.1   3.2      91.0            0.0  0.0
4  17.0   3.3      92.0            0.0  0.0
0    8179.056
1    8135.640
2    8107.128
3    8048.808
4    8043.624
Name: Wh, dtype: float64


In [7]:
train_x.describe()

,Temp,Wind,Moisture,Precipitation,Sun
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,20.243182,2.630682,71.022727,0.038636,0.290909
std,3.284229,1.082712,15.966903,0.220986,0.415404
min,14.800000,0.400000,39.000000,0.000000,0.000000
25%,17.600000,2.000000,59.500000,0.000000,0.000000
50%,20.100000,2.500000,74.500000,0.000000,0.000000
75%,22.175000,3.200000,84.500000,0.000000,0.625000
max,28.200000,5.500000,93.000000,1.900000,1.000000


In [8]:
train_x = train_x

최종 선택한 변수(들)이 왜 선택받았는지 간단히 알려주세요.<br>
답변 : 모든 설명변수들이 전력 사용량과 상관이 있을 것으로 생각됩니다. 맞추어야 할 레이블이 전력사용량이므로, 해당 일자의 날씨와 관련된 변수들은 모두 상관이 있을 것 같습니다.

## 전처리

In [9]:
from sklearn.preprocessing import StandardScaler

In [48]:
train_x.shape

(88, 5)

In [49]:
scaler = StandardScaler()
scaler = scaler.fit(train_x)
train_x = scaler.transform(train_x)

print(train_x[:10])

[[-0.80942259 -0.12139024  1.32132616  3.46504346 -0.70431807]
 [-0.77879955  0.25016944  1.2583377   1.18949253 -0.70431807]
 [-0.84004563  0.5288392   1.2583377  -0.17583803 -0.70431807]
 [-0.96253778  0.5288392   1.2583377  -0.17583803 -0.70431807]
 [-0.99316082  0.62172912  1.32132616 -0.17583803 -0.70431807]
 [-1.02378386  0.71461904  1.38431463 -0.17583803 -0.70431807]
 [-1.08502994  0.71461904  1.19534924  0.27927216 -0.70431807]
 [-1.02378386 -0.30717008  0.94339539 -0.17583803 -0.46220873]
 [-0.74817651  0.62172912  0.56546461 -0.17583803  0.02200994]
 [-0.28883093 -0.49294992  0.12454537 -0.17583803  0.99044729]]


# 모델링

## 학습

In [12]:
from sklearn.linear_model import LinearRegression

In [50]:
model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression()

## 추정

In [51]:
esti_y = model.predict(train_x)
print(esti_y)

[8133.35102092 8132.9800409  8118.12720673 8086.5289122  8088.47708072
 8090.42524923 8050.72345826 8009.47400661 8033.18574377 8051.05601898
 8055.07877999 8100.86259792 8160.73814354 8118.60604045 8133.02769749
 8163.14160222 8164.95517929 8139.32213582 8072.11769485 7979.7301432
 7930.80077424 7893.68494753 7889.45406585 7884.30947266 7881.10843268
 7862.67430841 7851.80133988 7825.57915687 7825.58377218 7776.24277719
 7805.44297041 7816.31132363 7851.00375857 7893.13198842 7874.57786071
 7933.65969827 7949.21698119 7910.98620377 8005.3153282  8012.39696646
 8057.28206958 8110.9859028  8075.26362616 8084.08614192 8088.90080577
 8101.03974964 8103.33095136 8113.98089013 8109.55070094 8121.6620493
 8135.86067657 8128.41565104 8106.77661354 8112.15734037 8067.61716329
 8117.21843163 8144.27188739 8193.45184971 8284.22660716 8335.17924559
 8342.05927131 8421.07568279 8442.02403855 8542.65881236 8537.38960042
 8552.73881489 8497.11648544 8398.88525791 8332.44627854 8270.03804017
 8251.71

# Loss 및 시각화

In [15]:
def mse(pred, true):
    return np.mean((pred-true)**2)

In [52]:
train_mse = mse(esti_y, train_y)
print(f"Linear {train_mse:.5f}")

Linear 2832.76020


# 예측

In [53]:
scaler = StandardScaler()
scaler = scaler.fit(test_x)
test_x = scaler.transform(test_x)

print(test_x)

[[ 2.04027348  1.38334315 -2.0781156   0.         -0.42008403]
 [ 1.35673008  1.93301593 -1.34895223  0.          1.26025208]
 [ 1.10816884  0.83367038 -1.05728689  0.          2.94058818]
 [ 0.42462544  1.05353949 -0.61978886  0.         -0.42008403]
 [ 0.23820452  0.17406305 -0.18229084  0.         -0.42008403]
 [-0.13463734 -0.48554428  0.25520718  0.         -0.42008403]
 [-0.44533888 -0.81534795  0.6927052   0.         -0.42008403]
 [-0.69390012 -1.03521706  0.98437055  0.         -0.42008403]
 [-0.81818073 -0.37560973  0.83853788  0.         -0.42008403]
 [-0.94246135 -1.03521706  0.83853788  0.         -0.42008403]
 [-1.06674197 -0.81534795  0.83853788  0.         -0.42008403]
 [-1.06674197 -0.81534795  0.83853788  0.         -0.42008403]]


In [54]:
pred_y = model.predict(test_x)
print(pred_y)

[8425.41855012 8312.59757107 8227.49480815 8196.88027223 8192.75013721
 8144.05498146 8116.67633118 8087.73734777 8046.89867454 8004.2656636
 7975.7304677  7975.7304677 ]


# 성능 평가

In [19]:
def score(train_mse, test_mse):
    print(f"{test_mse/train_mse: .4f}")

In [55]:
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")

Linear 240434.46017


In [56]:
score(train_mse,test_mse)

 84.8764


# 그 이후

## 비선형적 관계?

In [57]:
from sklearn.preprocessing import PolynomialFeatures

In [65]:
polynomial = PolynomialFeatures(2)
x_poly = polynomial.fit_transform(train_x)
x_poly.shape

(88, 21)

In [66]:
model.fit(x_poly, train_y)
esti_y = model.predict(x_poly)
print(esti_y)

[8179.59783647 8117.64343625 8068.26560008 8030.43937504 8027.61418213
 8024.67055675 8008.83241178 7968.84708619 8027.51761006 8058.92964532
 8106.73819305 8111.15038939 8151.93868142 8077.04739583 8059.84143355
 8097.76809146 8051.52453243 8074.7712812  8028.71983927 7978.59164995
 7948.883988   7954.78748192 7955.65456433 7938.55488619 7916.30117146
 7889.23053156 7873.77940125 7854.94891176 7847.08250067 7807.2650867
 7824.22492515 7851.01548359 7883.69772364 7930.51329198 7933.90428044
 7964.03559484 7935.63500743 7955.84813443 7998.27257077 8011.12236569
 8016.93542527 8082.06140696 8075.74254896 8082.68054697 8089.71597804
 8104.76550165 8105.24966269 8105.17090464 8113.79623138 8090.47562126
 8092.1367169  8079.70356881 8066.81544846 8066.45859234 8004.19005112
 8075.12182115 8161.73067634 8171.94468965 8280.39797014 8335.98379233
 8317.36825838 8405.40467551 8397.56372916 8522.13731343 8480.50239732
 8542.81366799 8619.29687186 8522.15202714 8350.08822535 8275.64120589
 8251.9

In [67]:
scaler = StandardScaler()
scaler = scaler.fit(test_x)
test_x = scaler.transform(test_x)
x_poly_test = polynomial.fit_transform(test_x)

In [68]:
x_poly_test.shape

(12, 21)

In [70]:
test_y.shape

(12,)

In [72]:
pred_y = model.predict(x_poly_test)
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")

Linear 274532.46599


비선형성을 반영하려고 해봤지만 성능이 오히려 악화됨 <br>
따라서 X에는 특별히 수정을 하지 않겠음

## 과적합의 문제??

In [73]:
from sklearn.linear_model import Ridge

In [80]:
for i in np.arange(0.1, 1, 0.1):
    
    ridge_model = Ridge(random_state=123, alpha = i).fit(train_x, train_y)

    ridge_esti_y = ridge_model.predict(train_x)
    ridge_pred_y = ridge_model.predict(test_x)

    train_mse_ridge = mse(ridge_esti_y, train_y)
    test_mse_ridge = mse(ridge_pred_y, test_y)
    print(f"Linear {test_mse_ridge:.5f}")

#     score(train_mse_ridge, test_mse_ridge)


Linear 240372.47380
Linear 240311.67594
Linear 240252.03642
Linear 240193.52596
Linear 240136.11620
Linear 240079.77959
Linear 240024.48944
Linear 239970.21982
Linear 239916.94559


릿지 규제를 사용한 경우 alpha = 0.9일 때 가장 성능이 좋았음 <br>
그리고 약간 성능이 좋아짐. 큰 차이는 아니었음

In [24]:
from sklearn.linear_model import Lasso

In [81]:
for i in np.arange(0.1, 1, 0.1):

    lasso_model = Lasso(random_state=123, alpha = i).fit(train_x, train_y)

    lasso_esti_y = lasso_model.predict(train_x)
    lasso_pred_y = lasso_model.predict(test_x)

    train_mse_lasso = mse(lasso_esti_y, train_y)
    test_mse_lasso = mse(lasso_pred_y, test_y)
    print(f"Linear {test_mse_lasso:.5f}")

#     score(train_mse_lasso, test_mse_lasso)


Linear 240379.88442
Linear 240325.06501
Linear 240269.60443
Linear 240215.92437
Linear 240161.28122
Linear 240107.06281
Linear 240053.26916
Linear 239999.90025
Linear 239946.95615


라쏘 규제의 경우에도 alpha = 0.9가 가장 좋았음. alpha값이 클수록 성능이 좋고 <br>
비용함수 값이 트레인 데이터와 테스트 데이터 간에 거의 100배 가까이 차이가 나므로 과대적합 문제가 있는 듯

In [26]:
from sklearn.linear_model import ElasticNet

In [82]:
for i in np.arange(0.1, 1, 0.1):
    
    print('alpha = ' + str(i))
    
    for j in np.arange(0.1, 1, 0.1):
        
        elastic_model = ElasticNet(random_state=123, alpha = i, l1_ratio = j).fit(train_x, train_y)

        elastic_esti_y = elastic_model.predict(train_x)
        elastic_pred_y = elastic_model.predict(test_x)

        train_mse_elastic = mse(elastic_esti_y, train_y)
        test_mse_elastic = mse(elastic_pred_y, test_y)
        
        print(f"Linear {test_mse_elastic:.5f}")

    #     score(train_mse_elastic, test_mse_elastic)

    

alpha = 0.1
Linear 237698.57056
Linear 237868.72208
Linear 238058.40364
Linear 238271.23240
Linear 238511.66932
Linear 238785.23601
Linear 239098.85287
Linear 239461.24284
Linear 239883.50336
alpha = 0.2
Linear 236681.80718
Linear 236851.83934
Linear 237047.01353
Linear 237275.23165
Linear 237530.97985
Linear 237843.46960
Linear 238241.44724
Linear 238749.11388
Linear 239417.20503
alpha = 0.30000000000000004
Linear 236104.80651
Linear 236270.83871
Linear 236459.66512
Linear 236680.12697
Linear 236945.49413
Linear 237276.29694
Linear 237654.42738
Linear 238212.03442
Linear 239019.90246
alpha = 0.4
Linear 235690.23700
Linear 235859.96927
Linear 236049.82044
Linear 236267.91182
Linear 236526.98888
Linear 236849.89163
Linear 237277.77098
Linear 237794.13750
Linear 238678.69551
alpha = 0.5
Linear 235353.28490
Linear 235530.86539
Linear 235726.80610
Linear 235948.23311
Linear 236206.80184
Linear 236524.69172
Linear 236944.91729
Linear 237510.35664
Linear 238382.13301
alpha = 0.6
Linear 23506

엘라스틱넷 회귀의 경우 alpha = 0.9, l1_ratio = 0.1인 경우가 가장 MSE(234352)가 낮았음 <br>
base 모형보다 MSE가 약 2% 정도 좋아졌으나, 규제 모델을 쓴 것이 의미는 없어보임 <br>

<br>
혹시나 해서 제곱항 추가와 규제를 동시에 해봤지만 오히려 base 모형보다 성능 악화되었음(MSE 대충 270000대)